In [35]:
import os

from dotenv import dotenv_values
from langchain.memory import ConversationBufferWindowMemory

env_vars = dotenv_values('.env')
os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY')

# return_messages=True : Chat 모델인 경우 
memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4  # 최근 대화 4개 저장 (최근 대화에 집중)
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)
add_message(5, 5)

memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}